# Grocery Product Recommendation using Apriori Algorithm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mlxtend import frequent_patterns 
from mlxtend.preprocessing import TransactionEncoder

In [34]:
df = pd.read_csv("store_data.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df.shape
# 7501 Orders with maximun 20 products bought together

(7501, 20)

In [63]:
type(df.values[1,4])

float

In [64]:
df= df.fillna(0)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Preprocessing 

In [67]:
records = []
for i in range(0, 7501):
    records.append([str(df.values[i, j]) for j in range(0, 20) if df.values[i, j] != 0])

In [114]:
# records

In [113]:
products = []
for i in range(1, 7501):
    for j in range(20):
        if str(df.values[i,j]) not in products:
            products.append(str(df.values[i,j]))
            
products.remove("0")

Total products:  120


In [ ]:
print("Total products: ", len(products))

In [69]:
te = TransactionEncoder()
te_array = te.fit(records).transform(records)

In [70]:
df_trans = pd.DataFrame(te_array, columns=te.columns_)
df_trans.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


### Useful insights from data

In [118]:
# List of top 10 products sold
df_trans.sum().sort_values(ascending=False).head(10)

mineral water        1788
eggs                 1348
spaghetti            1306
french fries         1282
chocolate            1229
green tea             991
milk                  972
ground beef           737
frozen vegetables     715
pancakes              713
dtype: int64

### Using Apriori Algorithm

In [94]:
freq_itemsets = frequent_patterns.apriori(df_trans,min_support=0.015,use_colnames=True)
freq_itemsets

,support,itemsets
0,0.020397,(almonds)
1,0.033329,(avocado)
2,0.033729,(brownies)
3,0.087188,(burgers)
4,0.030129,(butter)
...,...,...
148,0.020931,"(spaghetti, tomatoes)"
149,0.016531,"(spaghetti, turkey)"
150,0.015865,"(mineral water, spaghetti, chocolate)"
151,0.017064,"(mineral water, spaghetti, ground beef)"


In [95]:
rules = frequent_patterns.association_rules(freq_itemsets,metric='lift',min_threshold=1)
rules.sort_values(by='confidence',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
149,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255
182,"(milk, spaghetti)",(mineral water),0.035462,0.238368,0.015731,0.443609,1.861024,0.007278,1.368879
176,"(spaghetti, ground beef)",(mineral water),0.039195,0.238368,0.017064,0.435374,1.826477,0.007722,1.348914
141,(olive oil),(mineral water),0.065858,0.238368,0.027596,0.419028,1.757904,0.011898,1.310962
175,"(mineral water, ground beef)",(spaghetti),0.040928,0.174110,0.017064,0.416938,2.394681,0.009938,1.416470
...,...,...,...,...,...,...,...,...,...
177,(mineral water),"(spaghetti, ground beef)",0.238368,0.039195,0.017064,0.071588,1.826477,0.007722,1.034891
118,(mineral water),(herb & pepper),0.238368,0.049460,0.017064,0.071588,1.447397,0.005275,1.023835
171,(mineral water),"(spaghetti, chocolate)",0.238368,0.039195,0.015865,0.066555,1.698053,0.006522,1.029311
183,(mineral water),"(milk, spaghetti)",0.238368,0.035462,0.015731,0.065996,1.861024,0.007278,1.032691


In [96]:
rules[(rules['confidence']>0.16) & (rules['lift']>2)].head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
94,(shrimp),(frozen vegetables),0.071457,0.095321,0.016664,0.233209,2.446574,0.009853,1.179825
95,(frozen vegetables),(shrimp),0.095321,0.071457,0.016664,0.174825,2.446574,0.009853,1.125268
98,(tomatoes),(frozen vegetables),0.068391,0.095321,0.016131,0.235867,2.474464,0.009612,1.183930
99,(frozen vegetables),(tomatoes),0.095321,0.068391,0.016131,0.169231,2.474464,0.009612,1.121381
110,(ground beef),(herb & pepper),0.098254,0.049460,0.015998,0.162822,3.291994,0.011138,1.135410
111,(herb & pepper),(ground beef),0.049460,0.098254,0.015998,0.323450,3.291994,0.011138,1.332860
116,(spaghetti),(ground beef),0.174110,0.098254,0.039195,0.225115,2.291162,0.022088,1.163716
117,(ground beef),(spaghetti),0.098254,0.174110,0.039195,0.398915,2.291162,0.022088,1.373997
137,(soup),(milk),0.050527,0.129583,0.015198,0.300792,2.321232,0.008651,1.244861
174,"(mineral water, spaghetti)",(ground beef),0.059725,0.098254,0.017064,0.285714,2.907928,0.011196,1.262445


In [97]:
rules.antecedents.values

array([frozenset({'burgers'}), frozenset({'chocolate'}),
       frozenset({'eggs'}), frozenset({'burgers'}),
       frozenset({'french fries'}), frozenset({'burgers'}),
       frozenset({'green tea'}), frozenset({'burgers'}),
       frozenset({'milk'}), frozenset({'burgers'}),
       frozenset({'mineral water'}), frozenset({'burgers'}),
       frozenset({'burgers'}), frozenset({'spaghetti'}),
       frozenset({'eggs'}), frozenset({'cake'}),
       frozenset({'french fries'}), frozenset({'cake'}),
       frozenset({'mineral water'}), frozenset({'cake'}),
       frozenset({'spaghetti'}), frozenset({'cake'}),
       frozenset({'mineral water'}), frozenset({'chicken'}),
       frozenset({'spaghetti'}), frozenset({'chicken'}),
       frozenset({'eggs'}), frozenset({'chocolate'}),
       frozenset({'escalope'}), frozenset({'chocolate'}),
       frozenset({'french fries'}), frozenset({'chocolate'}),
       frozenset({'frozen vegetables'}), frozenset({'chocolate'}),
       frozenset({'green te

In [120]:
rules[rules["antecedents"] == frozenset({'shrimp'})].sort_values("confidence", ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
147,(shrimp),(mineral water),0.071457,0.238368,0.023597,0.330224,1.385352,0.006564,1.137144
163,(shrimp),(spaghetti),0.071457,0.174110,0.021197,0.296642,1.703760,0.008756,1.174209
46,(shrimp),(chocolate),0.071457,0.163845,0.017998,0.251866,1.537221,0.006290,1.117654
135,(shrimp),(milk),0.071457,0.129583,0.017598,0.246269,1.900474,0.008338,1.154811
94,(shrimp),(frozen vegetables),0.071457,0.095321,0.016664,0.233209,2.446574,0.009853,1.179825
